# PREPARATION DES DONNÉES

In [1]:
import pandas as pd

Dans ce Jupyter, nous allons importer le fichier revenus.csv et le modifier afin d'obtenir les différents fichiers de données nécessaire pour l'entraînement de nos modèles.

In [2]:
df_revenus = pd.read_csv('files/revenus.csv')
df_revenus.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


## DATA_V1

Le jeu de données data_V1 sera utilisée lors du premier cycle. Il est constitué du jeu de données revenus pour lequel nous avons supprimé les colonnes inutiles à la construction de nos modèles, soit fnlwgt et educational-num, et recodé ou regroupé les valeurs manquantes "?".

In [3]:
def drop_columns(df):
    """
    Drop les colonnes fnlwgt, educational-nul car inutiles
    """
    df = df.drop(["fnlwgt", "educational-num"], axis=1)
    return df

In [4]:
def regroupement_V1(df):
    """
    Applique les recodages :
    workclass : ? recodé en No_income_or_unknown
    native-country : ? recodé en Other_country
    occupation : ? regroupé avec la variable Other-service
    """
    # Regroupement des ? dans No_income_or_unknown
    df['workclass'] = df['workclass'].replace(
        {'\?' : 'No_income_or_unknown'}, regex=True)
    
    # Regroupement des ? dans Other_country
    df['native-country'] = df['native-country'].replace(
        {'\?': 'Other_country'}, regex=True)
    
    # Remplacement des ? dans Other-service 
    df['occupation'] = df['occupation'].replace(
        {'\?' : 'Other-service'}, regex=True)
    
    return df

In [5]:
df_V1 = drop_columns(df_revenus)
df_V1 = regroupement_V1(df_V1)
df_V1.to_csv('files/data/data_V1.csv', index=False)

## DATA_V2

Le jeu de données data_V2 sera utilsé lors du deuxième cycle. Ce fichier comporte les mêmes regroupements des valeurs manquantes que dans le premier cycle, nous y ajoutons des regroupements pour les autres variables afin d'obtenir des modalités plus représentatives en nombre d'invidivdus. Nous y appliquons enfin des classes optimisisées pour les variables quantitatives $\textbf{age}$ et $\textbf{hours-per-week}$.

In [6]:
def apply_opti_classes(df):
    """
    Applique les classes optimisées générées par le fichier optimisation_classes.py
    """
    serie_age = pd.read_csv('files/classes_opti/classes_opti_alexander_age.csv')
    serie_hpw = pd.read_csv('files/classes_opti/classes_opti_alexander_hours-per-week.csv') 
    df_classes_opti = pd.concat([serie_age, serie_hpw], axis=1)
    df.drop(['age', 'hours-per-week'], inplace=True, axis=1)
    df_classes_opti.columns = ['age', 'hours-per-week']
    df = pd.concat([df, df_classes_opti], axis=1)
    return df

In [7]:
def regroupement_V2(df):
    """
    Applique les regroupements de variables du deuxième cycle
    """
    # Regroupement des personnes d'ethnies autre que blanc et noir en "Other_race"
    df['race'] = df['race'].replace(
        {'Asian-Pac-Islander|Amer-Indian-Eskimo|Other': 'Other_race'}, regex=True)

    # Regroupement des personnes d'origines différentes en une cartégorie "Other"
    df['native-country'] = df['native-country'].replace(
        {'^(?!United-States$).+': 'Other_country'}, regex=True)

    # Regroupement des personnes mariés mais avec un conjoint dans l'armé avec les personnes mariés mais avec un conjoint absent pour diverse raisons
    df['marital-status'] = df['marital-status'].replace(
        {'Married-AF-spouse|Divorced|Widowed|Separated|Married-spouse-absent': 'Alone'}, regex = True)

    df['education'] = df['education'].replace(
        {'Preschool|1st-4th|5th-6th|7th-8th|9th|10th|11th|12th': 'Low-education',
        'Doctorate|Prof-school': "Graduation",
        "Assoc-acdm|Assoc-voc" : 'Assoc'}, regex=True)

    df['workclass'] = df['workclass'].replace(
        {'Local-gov|State-gov': 'State-Local-gov',
         'Never-worked|Without-pay|\?' : 'No_income_or_unknown'}, regex=True)

    df['occupation'] = df['occupation'].replace(
        {'\?|Priv-house-serv|Other-service|Handlers-cleaners': 'Occupation:Very-Low-income',
        'Farming-fishing|Machine-op-inspct|Adm-clerical': "Occupation:Low-income",
        'Transport-moving|Craft-repair' : 'Occupation:Mid-income',
        'Sales|Armed-Forces|Tech-support|Protective-serv|Protective-serv' :'Occupation:High-income',
        'Prof-specialty|Exec-managerial' : 'Occupation:Very-High-income'}, regex=True)
    
    return df

In [8]:
df_V2 = drop_columns(df_revenus)
df_V2 = apply_opti_classes(df_V2)
df_V2 = regroupement_V2(df_V2)
df_V2.to_csv('files/data/data_V2.csv', index=False)